# RAG OVER CODE - QUERY OPTIMIZATION

In [ ]:
import re
import os
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from pathlib import Path
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
import boto3
from langchain.chains import RetrievalQA
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema  import StrOutputParser
from langchain.schema.runnable  import RunnablePassthrough
from langchain_community.callbacks import get_openai_callback
from langchain.chains import HypotheticalDocumentEmbedder
from langchain_groq import ChatGroq
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from langchain.retrievers.multi_query import MultiQueryRetriever
from groq import Groq
import re
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)
import getpass
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display
from langchain_cohere import CohereEmbeddings


In [ ]:


queries = [
       "What are the steps required to set up a boilerplate project using the Emumba Plugin?",
        "Can you list some of the key features provided by the Emumba Plugin for React applications?",
        "What is the purpose of the generateFiles function in the project setup generator, and how does it use the options provided?",
        "Describe the role of addDependenciesToPackageJson in the project setup process.",
        "How does the ProjectSetupGeneratorSchema interface influence the behavior of the project setup generator?",
        "Explain how the project configuration is added to the workspace using addProjectConfiguration in the context of the setup process.",
        "Describe the process and the purpose of creating a test project in the beforeAll setup of the emumba-plugin tests.",
        "How does the test for emumba-plugin ensure that the plugin is properly installed and functional within a generated project?",
        "there's a function used to create a test project directory. Output the code snippet that showcases how this directory is created and initialized.",
        '''Given the following incomplete snippet, complete the function to add a specific dependency to the project's package.json. 
            Assume the function addDependenciesToPackageJson is already imported.
            function enhancePackageJson(tree: Tree, projectName: string) {
            // Add 'react-redux' as a dependency
            addDependenciesToPackageJson(tree, projectName, {
                'react-redux': '^7.2.0'
            }, {});
            // Complete the function to also add 'redux' as a dependency
        }'''
]

## Set up Ids for llm using bedrock service 

In [ ]:
groq_api_key="your-key"
llm = ChatGroq(
            groq_api_key=groq_api_key,
            model_name='llama3-70b-8192',
            temperature=0
    )    
embeddings = OllamaEmbeddings(model='nomic-embed-text')
client = Groq(
    api_key="your-key",
)
os.environ["OPENAI_API_KEY"]  = "your-key"
os.environ["COHERE_API_KEY"] =  "your-key"




## Extract content from Github Folder and Split using Langchain

In [ ]:


class Document:
    def __init__(self, content):
        self.page_content = content
        self.metadata = {} 

def read_files(directory_path):
    documents = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            file_path = Path(root) / file
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()
                    documents.append(Document(content)) 
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
    return documents

repo_path = 'data/Boilerplate'
documents = read_files(repo_path)
docs_texts = [d.page_content for d in documents]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)


docs = text_splitter.split_documents(documents)

documents = SimpleDirectoryReader('data/Boilerplate').load_data()


### Init Retriever

In [ ]:
vectorstore = Chroma.from_documents(
    documents=docs,
    collection_name="openai_embeds",
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
)        
retriever = vectorstore.as_retriever()

### STEP BACK PROMPTING

In [ ]:

system = """ we have a specific query then we firstly take a step back and try to get an answer for a generic query.
let's say we ask a specific question why does the login button on my website doesn't work for some users. 
and then a step back query would be what's the functionality of login button on a website for multiple users. So, genearte a step back query, and also return original query, your output format should be =  <stepbackquery> <original query>, here is the original query:"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
step_back = prompt | llm | StrOutputParser()

In [ ]:
def QA_chain(llm,retriever,queries,step_back):
    result=[]
    for i,query in enumerate(queries, start=0):
        file_path = 'positive_ground_truths.xlsx'
        df = pd.read_excel(file_path)
        question = (query)
        newqueries = step_back.invoke({"question": question})
        rag_template = """
        The following data comes from various files in a GitHub repository, which may contain information of any file extension. Your task is to search for an answer to a specific question within this data. Do not attempt to create an answer on your own. If you cannot find any reference to the query within the provided data, simply respond with, "There is no such reference to this."
    
        Data Context:
        {context}
    
        Question: {question}
    
        Answer:"""
    
        def process_context(source_documents):
            contexts = []
            for doc in source_documents:
                if hasattr(doc, 'page_content'):
                    contexts.append(doc.page_content)
                else:
                    contexts.append("Invalid document format")
            return contexts
    
        rag_prompt = ChatPromptTemplate.from_template(rag_template)
        qa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": rag_prompt}
        )
        
        response = qa({"query": newqueries})
                
        ground_truth = df.iloc[i]['ground truth']  
        contexts = process_context(response['source_documents'])
        result.append({'question': query, 'answer': response['result'], 'ground_truths': ground_truth,'contexts':contexts})
    return result
               
        
         

In [ ]:
result=QA_chain(llm,retriever,queries,step_back)


## Multi step 

In [ ]:

system = """ we have a query, for that given query you have to generate a follow up question
So, genearte a follow up question, and also return original query, your output format should be = <original query>,<Follow up query> here is the original query:"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
multi_step = prompt | llm | StrOutputParser()

In [ ]:
def multi_QA_chain(llm,retriever,queries,multi_step):
    for i,query in enumerate(queries, start=0):
        
        file_path = 'positive_ground_truths.xlsx'
        df = pd.read_excel(file_path)
        
        newqueries = multi_step.invoke({"question": (query)})
        print(newqueries)
        rag_template = """
        The following data comes from various files in a GitHub repository, which may contain information of any file extension. Your task is to search for an answer to a specific question within this data. Do not attempt to create an answer on your own. If you cannot find any reference to the query within the provided data, simply respond with, "There is no such reference to this."
    
        Data Context:
        {context}
    
        Question: {question}
    
        Answer:"""
    
        def process_context(source_documents):
            contexts = []
            for doc in source_documents:
                if hasattr(doc, 'page_content'):
                    contexts.append(doc.page_content)
                else:
                    contexts.append("Invalid document format")
            return contexts
    
        rag_prompt = ChatPromptTemplate.from_template(rag_template)
        qa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": rag_prompt}
        )
        
        response = qa({"query": newqueries})
        
        ground_truth = df.iloc[i]['ground truth']  
        contexts = process_context(response['source_documents'])
        result.append({'question': query, 'answer': response['result'], 'ground_truths': ground_truth,'contexts':contexts})
    return result

In [ ]:
result=multi_QA_chain(llm,retriever,queries,multi_step)


###  QUERY EXPANSION-MULTIQUERY

In [ ]:
vec_db = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings(model="text-embedding-3-large"))
base_retriever = retriever.as_retriever(search_kwargs={"k":3})
final_retriever = MultiQueryRetriever.from_llm(retriever, llm)


In [ ]:
def QA_chain(llm,final_retriever,queries):
    
    def process_context(source_documents):
        contexts = []
        for doc in source_documents:
            if hasattr(doc, 'page_content'):
                contexts.append(doc.page_content)
            else:
                contexts.append("Invalid document format")
        return contexts
    
    
    tmpl = """
        You are an assistant to answer a question from user with a context.

        Context:
        {context}

        Question:
        {question}

        The response should be presented as a list of key points, after creating the title of the content,
            formatted in HTML with appropriate markup for clarity and organization.
        """
    
    prompt = ChatPromptTemplate.from_template(tmpl)
    result=[]
    file_path = 'negative_ground_truths.xlsx'
    df = pd.read_excel(file_path)
    for i,query in enumerate(queries):
        qa = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=final_retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": prompt}
        )
        response= qa({"query": query})
        contexts = process_context(response['source_documents'])
        ground_truth = df.iloc[i]['ground truth']
        #result.append({'Query': query, 'Result': response['result'], 'GroundTruth': ground_truth, 'Context': contexts})
        result.append({'question': query, 'answer': response['result'], 'ground_truths': ground_truth,'contexts':contexts})

        
    return result

In [ ]:
result=QA_chain(llm,final_retriever,queries)


###  QUERY EXPANSION SUB QUERY


In [ ]:
def QA_chain(llm, retriever, queries):
    prompt_template = """
    The following data comes from various files in a GitHub repository, which may contain information of any file extension. Your task is to search for an answer to a specific question within this data. Do not attempt to create an answer on your own. If you cannot find any reference to the query within the provided data, simply respond with, "There is no such reference to this."

    Data Context:
    {context}

    Question: {question}

    Answer:"""

    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    def process_context(source_documents):
        contexts = []
        for doc in source_documents:
            if hasattr(doc, 'page_content'):
                contexts.append(doc.page_content)
            else:
                contexts.append("Invalid document format")
        return contexts

    def generate_sub_queries(query):
        sub_queries_prompt = f"Given the complex query '{query}', please break it down into only 2 constituent subqueries. Each subquery should focus on a specific aspect or component of the original query. Ensure that each subquery is formulated clearly and can be processed independently. Also return sub queries in an array like this [subquery1 ,subquery2, subquery3]"
    
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": sub_queries_prompt,
                }
            ],
            model="llama3-70b-8192",
        )
        sub_queries = chat_completion.choices[0].message.content
        
        pattern = r"\[(.*?)]"
        matches = re.search(pattern, sub_queries, re.DOTALL)

        if matches:
            sub_queries_list = [sub_query.strip() for sub_query in matches.group(1).split(',')]
            return sub_queries_list
        else:
            return [query]
            
            

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )

    result = []
    file_path = 'negative_ground_truths.xlsx'
    df = pd.read_excel(file_path)
    for i, query in enumerate(queries, start=0):
        sub_queries = generate_sub_queries(query)
    
        all_contexts = []

        for sub_query in sub_queries:
            response = qa({"query": sub_query})
            contexts = process_context(response['source_documents'])
            all_contexts.extend(contexts) 

        final_context = " ".join(all_contexts) 
        final_prompt=f"For given query : {query}, find answer for it from following context ; {final_context}"
        
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": final_prompt,
                }
            ],
            model="llama3-70b-8192",
        )
        final_response = chat_completion.choices[0].message.content

        ground_truth = df.iloc[i]['ground truth']
        
        result.append({'question': query, 'answer': final_response, 'ground_truths': ground_truth,'contexts':all_contexts})


    return result


In [ ]:
result=QA_chain(llm,retriever,queries)

### Hypothetical Document Embeddings (HyDE)

In [ ]:

embeddings = CohereEmbeddings()
documents = SimpleDirectoryReader("./data/Boilerplate").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)


In [ ]:
def hyderetriver(queries,query_engine):
    result=[]
    file_path = 'positive_ground_truths.xlsx'
    df = pd.read_excel(file_path)
    for i, query in enumerate(queries, start=0): 
        ground_truth = df.iloc[i]['ground truth']  
        response = hyde_query_engine.query(query)
        query_bundle = hyde(query)
        hyde_doc = query_bundle.embedding_strs
        result.append({'question': query, 'answer': str(response),'contexts':hyde_doc, 'ground_truths': ground_truth})
    return result

In [ ]:
result=hyderetriver(queries,query_engine)

### QUERY-REWRITING

In [ ]:

def QA_chain(llm,retreiver,questions):
    def _parse(text):
        return text.strip('"').strip("**")

    template = """Provide a better search query for \
    search engine to answer the given question, end \
    the queries with ’**’. Question: \
    {x} Answer:"""
    rewrite_prompt = ChatPromptTemplate.from_template(template)

    rewriter = rewrite_prompt | llm | StrOutputParser() | _parse
    rag_template = """
    The following data comes from various files in a GitHub repository, which may contain information of any file extension. Your task is to search for an answer to a specific question within this data. Do not attempt to create an answer on your own. If you cannot find any reference to the query within the provided data, simply respond with, "There is no such reference to this."

    Data Context:
    {context}

    Question: {question}

    Answer:"""

    def process_context(source_documents):
        contexts = []
        for doc in source_documents:
            if hasattr(doc, 'page_content'):
                contexts.append(doc.page_content)
            else:
                contexts.append("Invalid document format")
        return contexts

    rag_prompt = ChatPromptTemplate.from_template(rag_template)
    
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": rag_prompt}
    )
   
    result = []
    file_path = 'positive_ground_truths.xlsx'
    df = pd.read_excel(file_path)
    for i, query in enumerate(questions, start=0):
        rewrittenquery=rewriter.invoke({"x": query})
        response = qa({"query": rewrittenquery})
        print(response)
        contexts = process_context(response['source_documents'])
        ground_truth = df.iloc[i]['ground truth']
        result.append({
            'Query': query,
            'Result': response['result'],
            'GroundTruth': ground_truth,
            'Context': contexts
        })
    return result

In [ ]:
result=QA_chain(llm,retriever,queries)